#Pix2Text

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!pip install -qU langchain jq tiktoken langchain_community langchain_chroma langchain-huggingface huggingface-hub sentence_transformers
!pip install einops datasets bitsandbytes accelerate peft flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip uninstall -y transformers


Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2


In [4]:
!pip install transformers
!pip install sklearn
!pip install wandb
!pip install logging
!pip install --upgrade torch
!nvidia-smi

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 7.8 MB/s e

# Langchain

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings, HuggingFaceHubEmbeddings, HuggingFaceHubEmbeddings
from langchain_huggingface import HuggingFacePipeline

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
import bs4, getpass, os, tiktoken
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate
from transformers import BitsAndBytesConfig

# Load, chunk and index the contents of the blog.
#loader = TextLoader("/content/output/output.md")
#docs = loader.load()

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
#num_tokens_from_string(question, "cl100k_base")
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
#similarity = cosine_similarity(query_result, document_result)

In [10]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
#document = pages

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
vectorstore = Chroma(persist_directory="/content/drive/MyDrive/UWA/Sem 4/Capstone/Project/vector",embedding_function=hf)

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [35]:

#def format_docs(docs):
#    return "\n\n".join(" page number : "+ str(doc.metadata['page']) +" "+ doc.page_content for doc in docs)


# Function to format documents for inclusion in the context
def format_docs(docs):
    """
    Formats a list of documents into a string with content and metadata.

    Args:
        docs (list): A list of document objects with 'page_content' and 'metadata' attributes.

    Returns:
        str: A formatted string with each document's content and metadata.
    """
    return "\n\n".join(doc.page_content for doc in docs)

changing the format docs function to include the page number as part of the content for each chunk, not only in the metadata.

In [30]:

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline, BitsAndBytesConfig

model_name = "microsoft/Phi-3-medium-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',  # You can switch to torch.bfloat16 if needed
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    attn_implementation="flash_attention_2",
    trust_remote_code=True
)


tokenizer_config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Earlier we were using phi 3 mini which couldnt use complex prompts involving page number. after switching to phi 3 medium with flash attention 2, it can understand the prompt template better and extract page nmber and chapter number from chunks.

In [36]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

In [37]:
question = "What Ideal gas law?"

In [38]:
docs = retriever.get_relevant_documents(question)
total_tokens = 0
for doc in docs:
    # Extract the page content
    content = doc.page_content

    # Calculate the number of tokens in the page content
    tokens = num_tokens_from_string(content, "cl100k_base")
    total_tokens += tokens

print(total_tokens)

916


In [39]:
docs

[Document(metadata={'page': 18}, page_content='$$\nz={\\frac{p V} {n R T}}={\\frac{p} {\\rho_{M} R T}}=1 \\qquad\\qquad\\qquad\\qquad( 1. 5 6 )\n$$\n\nThis equation clearly is consistent with Eq. (1.11), which contains the definition of the absolute temperature. The limiting process indicated in Eq. (1.11) rigorously eliminates the virial coefficients to provide a precise definition; Eq. (1.56) covers a useful range of densities for practical calculations and is called the ideal gas law.\n\n## Partial pressures\n\nA useful quantity for dealing with the individual components in a gas mixture is partial pressure. The partial pressure of a component in a mixture, for example, component $A$ , is defined by the equation\n\n$$\np_{A} \\equiv P y_{A} \\qquad\\qquad\\qquad\\qquad\\qquad\\qquad( 1. 5 7 )\n$$\n\nP2E盟区CcR0m $A$ in mixture $y_{A}$ = mole fraction of component $A$ in mixture P = total pressure on mixture\n\nfalthearia resuresfor gien itue re aded, theresut i\n\n$$\np_{A}+p_{B}+p_{C

In [40]:
if total_tokens < 1000:
    template = """Answer the question and all the page numbers where this information is found based in the information provided in the context. Equations should be formatted in Latex.
    Providing all the relevant page numbers is essential.

    Context: {context}

    Response should be properly formatted to be easily read.

    Question: {question}

    """

else:
    template = """Do not answer the question; only provide the page numbers where they can find the information.

    Context: {context}
    Question: {question}

    """

prompt = ChatPromptTemplate.from_template(template)



In [ ]:
template = """Answer the question based on the following context along with page number:
{context}

Question: {question}
    """

prompt = ChatPromptTemplate.from_template(template)

In [41]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(question)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


"Human: Answer the question and all the page numbers where this information is found based in the information provided in the context. Equations should be formatted in Latex.\n    Providing all the relevant page numbers is essential.\n\n    Context: $$\nz={\\frac{p V} {n R T}}={\\frac{p} {\\rho_{M} R T}}=1 \\qquad\\qquad\\qquad\\qquad( 1. 5 6 )\n$$\n\nThis equation clearly is consistent with Eq. (1.11), which contains the definition of the absolute temperature. The limiting process indicated in Eq. (1.11) rigorously eliminates the virial coefficients to provide a precise definition; Eq. (1.56) covers a useful range of densities for practical calculations and is called the ideal gas law.\n\n## Partial pressures\n\nA useful quantity for dealing with the individual components in a gas mixture is partial pressure. The partial pressure of a component in a mixture, for example, component $A$ , is defined by the equation\n\n$$\np_{A} \\equiv P y_{A} \\qquad\\qquad\\qquad\\qquad\\qquad\\qquad(

Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

Question:
What Ideal gas law?

3

NOTE FOR FAILURE:
template = """Answer the question with the specific page numbers where the relevant information is found, based on the provided context. All equations should be formatted in LaTeX.

The output was generated the questions

    Context:
    {context}

    Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

    Question:
    {question}

    """
    
prompt = ChatPromptTemplate.from_template(template)
Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

Question:
What Ideal gas law?

1.56

What is the definition of the absolute temperature?

1.11

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56